In [ ]:
import os
os.chdir('/Users/bnowacki/Documents/Git Repositories/rapid-soh-estimation-from-short-pulses')

from rapid_soh_estimation.rapid_soh_estimation.config import *
from rapid_soh_estimation.rapid_soh_estimation.common_methods import *


cc_data = load_processed_data(data_type='cc')
pulse_data = load_processed_data(data_type='slowpulse')


In [2]:
def loss_function(y_true, y_pred):
	"""The loss function used for all optuna studies

	Args:
		y_true (MatrixLike | ArrayLike): Ground truth (correct) target values
		y_pred (MatrixLike | ArrayLike): Estimated target values

	Returns:
		_type_: The loss for this prediction (ie, prediction error)
	"""

	loss = mean_squared_error(y_true, y_pred)
	return loss

class OptunaHyperParamOptimization:
	def __init__(self, X, y, splits, loss_fnc, random_state):
		self.X = X
		if len(self.X.shape) == 1: self.X = self.X.reshape(-1,1)
		self.y = y
		if len(self.y.shape) == 1: self.y = self.y.reshape(-1,1)
		self.splits = splits
		self.loss_fnc = loss_fnc
		self.random_state = random_state

	def __call__(self, trial:optuna.trial.Trial):
		return None

class OptunaHyperParamOptimization_Ridge(OptunaHyperParamOptimization):
	def __call__(self, trial:optuna.trial.Trial):
		# average the loss over all cross-validation splits
		total_loss = 0
		for train_idxs, test_idxs in self.splits:
			# get suggest alpha value from Optuna search space
			alpha = trial.suggest_float('alpha', 0.0, 1e4)
			# create Ridge model
			model = Ridge(
				alpha=alpha,
				random_state=self.random_state
			)

			# standardize input and output data (using only the training data to create the scaler)
			scaler_X = StandardScaler().fit(self.X[train_idxs])
			scaler_y = StandardScaler().fit(self.y[train_idxs])
			X_sc = scaler_X.transform(self.X)
			y_sc = scaler_y.transform(self.y)

			# fit model to scaled input and output data
			model.fit(X_sc[train_idxs], y_sc[train_idxs])
			
			# get predictions
			y_pred_sc = model.predict(X_sc[test_idxs])

			# add loss to total
			total_loss += self.loss_fnc(y_sc[test_idxs], y_pred_sc)

		# return average cross-validation loss
		return total_loss / len(self.splits)

class OptunaHyperParamOptimization_Lasso(OptunaHyperParamOptimization):
	def __call__(self, trial:optuna.trial.Trial):
		# average the loss over all cross-validation splits
		total_loss = 0
		for train_idxs, test_idxs in self.splits:
			# get suggested model parameters from Optuna search space
			alpha = trial.suggest_float('alpha', 0, 1.0)

			# create Lasso model
			model = Lasso(
				alpha=alpha,
				random_state=self.random_state
			)

			# standardize input and output data (using only the training data to create the scaler)
			scaler_X = StandardScaler().fit(self.X[train_idxs])
			scaler_y = StandardScaler().fit(self.y[train_idxs])
			X_sc = scaler_X.transform(self.X)
			y_sc = scaler_y.transform(self.y)

			# fit model to scaled input and output data
			model.fit(X_sc[train_idxs], y_sc[train_idxs])
			
			# get predictions
			y_pred_sc = model.predict(X_sc[test_idxs])

			# add loss to total
			total_loss += self.loss_fnc(y_sc[test_idxs], y_pred_sc)

		# return average cross-validation loss
		return total_loss / len(self.splits)

class OptunaHyperParamOptimization_ElasticNet(OptunaHyperParamOptimization):
	def __call__(self, trial:optuna.trial.Trial):
		# average the loss over all cross-validation splits
		total_loss = 0
		for train_idxs, test_idxs in self.splits:
			# get suggested model parameters from Optuna search space
			alpha = trial.suggest_float('alpha', 0.0, 1e3)
			l1_ratio = trial.suggest_float('l1_ratio', 0.01, 1.0)

			# create ElasticNet model
			model = ElasticNet(
				alpha=alpha, 
				l1_ratio=l1_ratio, 
				random_state=self.random_state,
			)

			# standardize input and output data (using only the training data to create the scaler)
			scaler_X = StandardScaler().fit(self.X[train_idxs])
			scaler_y = StandardScaler().fit(self.y[train_idxs])
			X_sc = scaler_X.transform(self.X)
			y_sc = scaler_y.transform(self.y)

			# fit model to scaled input and output data
			model.fit(X_sc[train_idxs], y_sc[train_idxs])
			
			# get predictions
			y_pred_sc = model.predict(X_sc[test_idxs])

			# add loss to total
			total_loss += self.loss_fnc(y_sc[test_idxs], y_pred_sc)

		# return average cross-validation loss
		return total_loss / len(self.splits)

class OptunaHyperParamOptimization_RandomForestRegressor(OptunaHyperParamOptimization):
	def __call__(self, trial:optuna.trial.Trial):
		# average the loss over all cross-validation splits
		total_loss = 0
		for train_idxs, test_idxs in self.splits:
			# get suggested model parameters from Optuna search space
			n_estimators = trial.suggest_int('n_estimators', 5, 500)
			min_samples_split = trial.suggest_float('min_samples_split', 0.01, 1.0)
			min_samples_leaf = trial.suggest_float('min_samples_leaf', 0.01, 1.0)
			max_features = trial.suggest_float('max_features', 0.01, 1.0)
			max_samples = trial.suggest_float('max_samples', 0.01, 1.0)

			# create RandomForestRegressor model
			model = RandomForestRegressor(
				n_estimators=n_estimators, 
				min_samples_split=min_samples_split,
				min_samples_leaf=min_samples_leaf,
				max_features=max_features,  
				max_samples=max_samples,
				n_jobs=-1,
			)

			# standardize input and output data (using only the training data to create the scaler)
			scaler_X = StandardScaler().fit(self.X[train_idxs])
			scaler_y = StandardScaler().fit(self.y[train_idxs])
			X_sc = scaler_X.transform(self.X)
			y_sc = scaler_y.transform(self.y)

			# fit model to scaled input and output data
			model.fit(X_sc[train_idxs], y_sc[train_idxs])
			
			# get predictions
			y_pred_sc = model.predict(X_sc[test_idxs])

			# add loss to total
			total_loss += self.loss_fnc(y_sc[test_idxs], y_pred_sc)

		# return average cross-validation loss
		return total_loss / len(self.splits)

def print_optuna_study_results(study:optuna.Study):
	"""Prints the best loss and parameters for a given Optuna study"""
	
	print()
	print('*'*100)
	print(f'  Study: {study.study_name}')
	print('*'*100)
	print('  Best Loss: ', study.best_trial.value)
	print('  Best Params: ')
	for k,v in study.best_trial.params.items():
		print(f'    {k}: {v}')
	print('*'*100)
	print()

def perform_hyperparam_optimization(params:dict):
	for key,val in params.items():
		study = optuna.create_study(
			study_name=key,
			direction='minimize', 
			sampler=optuna.samplers.TPESampler(seed=random_state),)
		study.optimize(
			func = val['objective'],
			n_trials = val['n_trials'],
			n_jobs=-1)
		if val['save_results']:
			pickle.dump(
				study, 
				open(dir_hyperparam_results.joinpath(f"{val['filename']}.pkl"), 'wb'),
				protocol=pickle.HIGHEST_PROTOCOL)
		else:
			print_optuna_study_results(study)




random_state = 1

# use only q_dchg target (drop resistances)
# get modeling data (charge pulse only)
all_data = deepcopy(pulse_data)
idxs = np.where((all_data['pulse_type'] == 'chg'))
for k in all_data.keys():
	all_data[k] = all_data[k][idxs]
all_data['voltage_rel'] = np.asarray([v - v[0] for v in all_data['voltage']])
modeling_data = create_modeling_data(all_data=all_data, input_feature_keys=['voltage_rel'])
modeling_data['model_output'] = modeling_data['model_output'][:,0]

cv_splitter = Custom_CVSplitter(n_splits=3, split_type='group_id', rand_seed=random_state)
cv_splits = list(cv_splitter.split(
	X = modeling_data['model_input'], 
	y = modeling_data['model_output'], 
	cell_ids = modeling_data['cell_id']))

dir_hyperparam_results = dir_repo_main.joinpath("results", 'hyperparameter_optimization')
dir_hyperparam_results.mkdir(exist_ok=True, parents=True)

# key = study_name
hyperparam_opt_params = {
	'Hyperparameters_Ridge': {
		'objective':
			OptunaHyperParamOptimization_Ridge(
				X=modeling_data['model_input'], 
				y=modeling_data['model_output'], 
				splits=cv_splits, 
				loss_fnc=loss_function, 
				random_state=random_state
			),
		'n_trials':1000,
		'save_results':True,
		'filename':'hyperparam_study_ridge_onlySOH_byCell'
	},
	'Hyperparameters_Lasso': {
		'objective':
			OptunaHyperParamOptimization_Lasso(
				X=modeling_data['model_input'], 
				y=modeling_data['model_output'], 
				splits=cv_splits, 
				loss_fnc=loss_function, 
				random_state=random_state
			),
		'n_trials':1000,
		'save_results':True,
		'filename':'hyperparam_study_lasso_onlySOH'
	},
	'Hyperparameters_ElasticNet': {
		'objective':
			OptunaHyperParamOptimization_ElasticNet(
				X=modeling_data['model_input'], 
				y=modeling_data['model_output'], 
				splits=cv_splits, 
				loss_fnc=loss_function, 
				random_state=random_state
			),
		'n_trials':1000,
		'save_results':True,
		'filename':'hyperparam_study_elasticnet_onlySOH'
	},
	'Hyperparameters_RandomForestRegressor': {
		'objective':
			OptunaHyperParamOptimization_RandomForestRegressor(
				X=modeling_data['model_input'], 
				y=modeling_data['model_output'], 
				splits=cv_splits, 
				loss_fnc=loss_function, 
				random_state=random_state
			),
		'n_trials':2000,
		'save_results':True,
		'filename':'hyperparam_study_randomforest_onlySOH'
	},

}

# perform_hyperparam_optimization(hyperparam_opt_params)

In [ ]:
dir_hyperparam_results = dir_repo_main.joinpath("results", 'hyperparameter_optimization')
dir_hyperparam_results.mkdir(exist_ok=True, parents=True)

# print best hyperparameters for all studies in dir_hyperparam_results
for file_res in dir_hyperparam_results.glob('*_onlySOH.pkl'):
	study = pickle.load(open(file_res, 'rb'))
	print_optuna_study_results(study)

In [ ]:
def plot_feature_importance(dir_hyperparam_results:Path, dir_save:Path=None):
	"""Plots the feature importance of the pulse signal using linear models and a random forest regressor.

	Args:
		dir_hyperparam_results (Path): Path object to folder containing hyperparameter results
		dir_save (Path, optional): If provided, the plot will be saved in this folder. Defaults to None.
	"""

	plot_params = {
		'ridge': {'legend_key':'Ridge', 'color':'C0',},
		'lasso': {'legend_key':'Lasso', 'color':'C1',},
		'elasticnet': {'legend_key':'ElasticNet', 'color':'C2',},
		'randomforest': {'legend_key':'RandomForest', 'color':'C4',},
	}

	#region: prepare data for plotting
	#region: create modeling dataset (charge pulse, all SOCs)
	# load pulse and cc data
	cc_data = load_processed_data(data_type='cc')
	pulse_data = load_processed_data(data_type='slowpulse')
	# create modeling data
	all_data = deepcopy(pulse_data)
	idxs = np.where((all_data['pulse_type'] == 'chg'))
	for k in all_data.keys():
		all_data[k] = all_data[k][idxs]
	all_data['voltage_rel'] = np.asarray([v - v[0] for v in all_data['voltage']])
	modeling_data = create_modeling_data(all_data=all_data, input_feature_keys=['voltage_rel'])
	X = modeling_data['model_input']
	y = modeling_data['model_output'][:,0].reshape(-1,1)
	#endregion

	#region: get CV splits
	cv_splitter = Custom_CVSplitter(n_splits=3, split_type='group_id', rand_seed=random_state)
	cv_splits = list(cv_splitter.split(
		X = modeling_data['model_input'], 
		y = modeling_data['model_output'][:,0], 
		cell_ids = modeling_data['cell_id']))
	#endregion

	# use a fixed split for plotting
	train_idxs, test_idxs = cv_splits[0]

	# standardize data using only training samples
	scaler_X = StandardScaler().fit(X[train_idxs])
	scaler_y = StandardScaler().fit(y[train_idxs])
	X_sc = scaler_X.transform(X)
	y_sc = scaler_y.transform(y)
	#endregion

	fig = plt.figure(figsize=(6.25,3.75))
	gs = GridSpec(nrows=2, ncols=2, height_ratios=[3,2], width_ratios=[1,1])
	axes = [
		fig.add_subplot(gs[0, 0]), fig.add_subplot(gs[0, 1]),
		fig.add_subplot(gs[1, 0]), fig.add_subplot(gs[1, 1])
	]
	# axes[2].sharex(axes[0])
	# axes[3].sharex(axes[1])
	axes[0].set_xticklabels([])
	axes[1].set_xticklabels([])

	for model_type in plot_params.keys():
		file_study = list(dir_hyperparam_results.glob(f'*{model_type}_onlySOH.pkl*'))[0]
		study = pickle.load(open(file_study, 'rb'))
		# create optimal model from saved parameters values
		model = None
		if model_type == 'ridge': model = Ridge(**study.best_trial.params)
		if model_type == 'lasso': model = Lasso(**study.best_trial.params)
		if model_type == 'elasticnet': model = ElasticNet(**study.best_trial.params)
		if model_type == 'randomforest': model = RandomForestRegressor(**study.best_trial.params)
		# fit model to training data
		model.fit(X_sc[train_idxs], y_sc[train_idxs])

		x_vals = None
		if model_type == 'randomforest':
			x_vals = model.feature_importances_
			axes[1].plot(
				x_vals*100, '-', linewidth=2,
				color=plot_params[model_type]['color'],
				label=plot_params[model_type]['legend_key'])
		else:
			x_vals = model.coef_
			if len(x_vals.shape) == 2: x_vals = np.ravel(x_vals)
			axes[0].plot(
				x_vals, '-', linewidth=2,
				color=plot_params[model_type]['color'],
				label=plot_params[model_type]['legend_key'])

		if model_type == 'ridge':
			# region: overlay feature importance on pulse profile
			pulse_v = X[train_idxs][0,:]*1000
			axes[2].plot(pulse_v, 'k', linewidth=2)
			ybounds = [np.min(pulse_v), np.max(pulse_v)]
			ybounds[0] -= (ybounds[1]-ybounds[0])*0.05
			ybounds[1] += (ybounds[1]-ybounds[0])*0.05

			norm_coeff = abs(x_vals)
			norm_coeff = (norm_coeff - np.min(norm_coeff)) / (np.max(norm_coeff) - np.min(norm_coeff))
			for i in np.arange(0,100,1):
				axes[2].fill_betweenx(
					ybounds, i-0.5, i+0.5, 
					color=plot_params[model_type]['color'], alpha=norm_coeff[i])
			axes[2].set_xlim([-5,104])
			axes[2].set_ylim(ybounds)
			#endregion
		elif model_type == 'randomforest':
			# region: overlay feature importance on pulse profile
			pulse_v = X[train_idxs][0,:]*1000
			axes[3].plot(pulse_v, 'k', linewidth=2)
			ybounds = [np.min(pulse_v), np.max(pulse_v)]
			ybounds[0] -= (ybounds[1]-ybounds[0])*0.05
			ybounds[1] += (ybounds[1]-ybounds[0])*0.05

			norm_coeff = abs(x_vals)
			norm_coeff = (norm_coeff - np.min(norm_coeff)) / (np.max(norm_coeff) - np.min(norm_coeff))
			for i in np.arange(0,100,1):
				axes[3].fill_betweenx(
					ybounds, i-0.5, i+0.5, 
					color=plot_params[model_type]['color'], alpha=norm_coeff[i])
			axes[3].set_xlim([-5,104])
			axes[3].set_ylim(ybounds)
			#endregion
	#region: set labels
	axes[0].plot([0,99],[0,0], '--', color='k')
	axes[0].set_ylabel("Model Coefficients [-]")
	axes[0].set_title('Linear Models')
	axes[0].legend(fontsize=8, loc='lower right')

	axes[1].set_ylabel("Feature Importance [%]")
	axes[1].set_title('Random Forest')

	axes[2].set_xlabel("Time [s]")
	axes[2].set_ylabel("Rel. Voltage [mV]")
	axes[3].set_xlabel("Time [s]")
	axes[3].set_ylabel("Rel. Voltage [mV]")
	fig.align_ylabels()
	#endregion
	
	fig.tight_layout(h_pad=0.5, w_pad=1.2)
	if dir_save is not None:
		filename = dir_save.joinpath("FeatureImportance_Pulse.pdf")
		fig.savefig(filename, dpi=300)
	plt.show()

dir_save = dir_figures.joinpath("raw", "Feature Importance")
dir_save.mkdir(exist_ok=True, parents=True)
plot_feature_importance(
	dir_hyperparam_results = dir_repo_main.joinpath("results", 'hyperparameter_optimization'),
	dir_save = dir_save
)

In [ ]:
def plot_model_performance(dir_hyperparam_results:Path, dir_save:Path=None):
	"""Plots the baseline model perforamnce for linear models, random forest regressor, and MLP

	Args:
		dir_hyperparam_results (Path): Path object to folder containing hyperparameter results
		dir_save (Path, optional): If provided, the plot will be saved in this folder. Defaults to None.
	"""

	def _evaluate_model(model, X, y, cv_splits) -> dict:
		"""Evaluates the train and test accuracy of a given model

		Args:
			model (-): A model to be tested. Must have .fit() and .predict() functions
			X (_type_): Training data
			y (_type_): Test data
			cv_splits (_type_): A list of indices for train/test splits at each CV fold

		Returns:
			dict: Returns the average and std of the train and test error. Follows the following structure: {'train':(avg, std), 'test':(avg, std)}
		"""

		# average the loss over all cross-validation splits
		test_err = []
		train_err = []
		for train_idxs, test_idxs in cv_splits:
			# standardize input and output data (using only the training data to create the scaler)
			scaler_X = StandardScaler().fit(X[train_idxs])
			scaler_y = StandardScaler().fit(y[train_idxs])
			X_sc = scaler_X.transform(X)
			y_sc = scaler_y.transform(y)
			# fit model to scaled input and output data
			model.fit(X_sc[train_idxs], y_sc[train_idxs])
			
			# get test and train predictions
			yhat_train = model.predict(X_sc[train_idxs])
			yhat_test = model.predict(X_sc[test_idxs])
			train_err.append( mean_absolute_percentage_error(y_sc[train_idxs], yhat_train))
			test_err.append( mean_absolute_percentage_error(y_sc[test_idxs], yhat_test))

		# return average and std of mape
		return {
			'test':(np.average(np.asarray(test_err)), np.std(np.asarray(test_err))),
			'train':(np.average(np.asarray(train_err)), np.std(np.asarray(train_err)))
		}
	
	def _create_mlp_model(n_hlayers:int, n_neurons:int, act_fnc:str, opt_fnc:str, learning_rate:float, input_shape=(100,), output_shape=(7,)) -> keras.models.Sequential:
		"""Builds a Keras neural network model (MLP) using the specified parameters. The model is optimized for accuracy. Make sure model outputs (if multiple target) are normalized, otherwise optimization will be biased towards one target variable.

		Args:
			n_hlayers (int): Number of fully-connected hidden layers
			n_neurons (int): Number of neurons per hidden layer
			act_fnc (str): Activation function to use (\'tanh\', \'relu\', etc)
			opt_fnc (str): {\'sgd\', \'adam\'} Optimizer function to use 
			learning_rate (float): Learning rate
			input_shape (int, optional): Input shape of model. Defaults to (100,).
			output_shape (int, optional): Output shape of model. Default to (7,).
		Raises:
			ValueError: _description_

		Returns:
			keras.models.Sequential: compiled Keras model
		"""

		# add input layer to Sequential model
		model = keras.models.Sequential()
		model.add( keras.Input(shape=input_shape) )

		# add hidden layers
		for i in range(n_hlayers):
			model.add( keras.layers.Dense(units=n_neurons, activation=act_fnc) )
			
		# add output layer
		model.add( keras.layers.Dense(output_shape[0]) )

		# compile model with chosen metrics
		opt = None
		if opt_fnc == 'adam':
			opt = keras.optimizers.Adam(learning_rate=learning_rate)
		elif opt_fnc == 'sgd':
			opt = keras.optimizers.SGD(learning_rate=learning_rate)
		else:
			raise ValueError("opt_func must be either \'adam\' or \'sgd\'")

		model.compile(
			optimizer=opt,
			loss=keras.losses.mean_squared_error,      
			# make sure to normalize all outputs, otherwise DCIR values will drastically skew MSE reading compared to error of predicted SOH
			metrics=['accuracy'] )
		return model

	dir_hyperparam_results = dir_repo_main.joinpath("results", 'hyperparameter_optimization')
	plot_params = {
		'ridge': {'legend_key':'Ridge', 'color':'C0',},
		'lasso': {'legend_key':'Lasso', 'color':'C1',},
		'elasticnet': {'legend_key':'ElasticNet', 'color':'C2',},
		'randomforest': {'legend_key':'RF', 'color':'C4',},
		'mlp_chg':{'legend_key':'MLP', 'color':'C5',},
	}

	#region: prepare data for plotting
	#region: create modeling dataset (charge pulse, all SOCs)
	# load pulse and cc data
	cc_data = load_processed_data(data_type='cc')
	pulse_data = load_processed_data(data_type='slowpulse')
	# create modeling data
	all_data = deepcopy(pulse_data)
	idxs = np.where((all_data['pulse_type'] == 'chg'))
	for k in all_data.keys():
		all_data[k] = all_data[k][idxs]
	all_data['voltage_rel'] = np.asarray([v - v[0] for v in all_data['voltage']])
	modeling_data = create_modeling_data(all_data=all_data, input_feature_keys=['voltage_rel'])
	X = modeling_data['model_input']
	y = modeling_data['model_output'][:,0].reshape(-1,1)
	#endregion

	#region: get CV splits
	cv_splitter = Custom_CVSplitter(n_splits=3, split_type='group_id', rand_seed=random_state)
	cv_splits = list(cv_splitter.split(
		X = modeling_data['model_input'], 
		y = modeling_data['model_output'][:,0], 
		cell_ids = modeling_data['cell_id']))
	#endregion
	#endregion

	fig, ax = plt.subplots(figsize=(4,2))
	labels = []
	for i, model_type in enumerate(list(plot_params.keys())):
		file_study = list(dir_hyperparam_results.glob(f'*{model_type}_onlySOH.pkl*'))[0]
		study = pickle.load(open(file_study, 'rb'))
		# create optimal model from saved parameters values
		model = None
		if model_type == 'ridge': model = Ridge(**study.best_trial.params)
		elif model_type == 'lasso': model = Lasso(**study.best_trial.params)
		elif model_type == 'elasticnet': model = ElasticNet(**study.best_trial.params)
		elif model_type == 'randomforest': model = RandomForestRegressor(**study.best_trial.params)
		elif model_type == 'mlp_chg': model = _create_mlp_model(**study.best_trial.params, output_shape=(1,))
		
		# plot avg model error 
		res = _evaluate_model(model, X, y, cv_splits)
		
		ax.bar(i-0.15, res['train'][0], width=0.3, align='center', color='C0')
		ax.bar(i+0.15, res['test'][0], width=0.3, align='center', color='C1')
		# ax.errorbar(i-0.15, res['train'][0], res['train'][1], capsize=2, color='k')
		# ax.errorbar(i+0.15, res['test'][0], res['test'][1], capsize=2, color='k')
		labels.append(plot_params[model_type]['legend_key'])

	ax.set_xlabel("Model Name")
	ax.set_xticks(np.arange(0,len(labels),1), labels=labels, fontsize=8)
	ax.set_ylabel("MAPE [%]")
	ax.set_ylim([0,2.8])
	ax.set_yticks(np.arange(0,2.8,0.5), np.arange(0,2.8,0.5))
	ax.legend(['Train Error', 'Test Error'], ncols=2, loc='upper center', fontsize=8)
	if dir_save is not None:
		filename = dir_save.joinpath("ModelPerformanceBaseline_Pulse.pdf")
		fig.savefig(filename, dpi=300, bbox_inches='tight')
	plt.show()

dir_save = dir_figures.joinpath("raw", "Feature Importance")
dir_save.mkdir(exist_ok=True, parents=True)
plot_model_performance(
	dir_hyperparam_results = dir_repo_main.joinpath("results", 'hyperparameter_optimization'),
	dir_save = dir_save
)